# Doing Moles
A notebook to help with moles calculations

## Variable Names
**s**   - Substance  
**n**   - Moles  
**m**   - Mass in g  
**V**   - Volume in dm^3  
**c**   - Concentration in mol/dm^3  
**Mr**  - Molecular Mass in g/mol  

## Setup

In [1]:
from ipywidgets import widgets
from collections import namedtuple, OrderedDict

In [2]:
class MolesStep:
    def __init__(self, name, ddDesc):
        '''
           name = The name of the step
           ddDesc = The message next to the dropdown
        '''
        self.name = name
        self.ddDesc = ddDesc
        self.conversionMethods = OrderedDict()
    
    def register_conversion_method(self, functionName, parameters):
        '''
           Method for generating decorator to register a conversion function.
           
           functionName = The name of the function in the dropdown
           parameters = tuple containing each parameter as (name, type) where type is str, int or float
        '''
        for param in parameters:
            if param[1] not in (str, float, int):
                raise ValueError('Invalid parameter type ({t}) encountered when registering {f}'.format(t=param[1],
                                                                                                        f=functionName))
        def method_registration_decorator(f):
            self.conversionMethods[functionName] = (f, parameters) 
        return method_registration_decorator
    
    def make_box(self, *nonUserParams):
        '''
           Method to make a box to display for user input
        '''
        self.w_title = widgets.HTML(value='<h3>{}<h3>'.format(self.name))
        options = list(self.conversionMethods.keys())
        self.w_dropdown = widgets.Dropdown(
            options=options,
            value = options[0],
            description = self.ddDesc
        )
        self.nonUserParams = nonUserParams
        self.w_param_box = widgets.Box()
        self.edit_param_box()
        self.w_dropdown.observe(self.edit_param_box)
        if len(options)>1:
            self.w_box = widgets.Box(
                children=[self.w_title, self.w_dropdown, self.w_param_box]
            )
        else:
            self.w_box = widgets.Box(
                children=[self.w_title, self.w_param_box]
            )
        return self.w_box
    
    def edit_param_box(self, *args):
        w_params = []
        for desc, typ in self.conversionMethods[self.w_dropdown.value][1]:
            if desc not in self.nonUserParams:
                if typ is int:
                    w_params.append(widgets.IntText(
                            value=1,
                            description=desc
                        ))
                elif typ is float:
                    w_params.append(widgets.FloatText(
                            value=1,
                            description=desc
                        ))
                elif typ is str:
                    w_params.append(widgets.Text(
                            description=desc
                        ))
        self.w_param_box.children = w_params
    
    def run(self, nonUserParams={}):
        '''
           Method to run the actual calculations
           
           nonUserParams = Dictionary containing {'param':'non user entered value'}
        '''
        userParams = {w_param.description : w_param.value for w_param in self.w_param_box.children}
        currentConversionMethod = self.conversionMethods[self.w_dropdown.value]
        allParams = []
        for desc, typ in currentConversionMethod[1]:
            if desc in nonUserParams:
                allParams.append(nonUserParams[desc])
            else:
                allParams.append(userParams[desc])
        return currentConversionMethod[0](*allParams)

## Doing Stuff with Substances

In [3]:
MolesPrepStep = MolesStep('Enter substance names',
                          'You can\'t change this')

In [4]:
@MolesPrepStep.register_conversion_method(
    functionName='Useless',
    parameters=(('Name 1',str),('Name 2',str))
)
def placeholder(s1, s2):
    return s1, s2  # Could return Mr in the future

## Calculate Original Moles from Data

In [5]:
MolesStep1 = MolesStep('Step 1: Get moles of substance you know',
                       'What you know:')

In [6]:
@MolesStep1.register_conversion_method(
    functionName='Solution',
    parameters=(('Name',str),('Concentration',float),('Volume',float))
)
def calculate_moles_from_solution(s, c, V):
    '''n = cV'''
    n = c*V
    print('n({s}) = cV = {c} x {V} = {n:.8g}mol'.format(s=s, c=c, V=V, n=n))
    return n

In [7]:
@MolesStep1.register_conversion_method(
    functionName='Gas',
    parameters=(('Name',str),('Volume',float))
)
def calculate_moles_from_gas(s, V):
    '''n = V/24 at 25°C (298K) and 1atm'''
    n = V/24
    print('n({s}) = V/24 = {V} / 24 = {n:.8g}mol'.format(s=s, V=V, n=n))
    return n

## Calculate Final Moles from Ratio

In [8]:
MolesStep2 = MolesStep('Step 2: Get the moles of the substance you want from the ratio',
                       'You can\'t change this:')

In [9]:
@MolesStep2.register_conversion_method(
    functionName='You can only pick this method.',
    parameters=(('Name 1',str), ('Name 2',str), 
                ('Ratio 1',int), ('Ratio 2',int),
                ('Moles 1',float)
               )
)
def get_moles_from_ratio(s1, s2, r1, r2, n1):
    '''1 mol X:1 mol Y'''
    tot_r = r2/r1
    n2 = n1*tot_r
    print(('From eqn {r1} mol {s1}:{r2} mol {s2}\n'+
           'TF n({s2}) = {n1:.8g} x ({r2}/{r1}) = {n2:.8g}mol').format(s1=s1, s2=s2, r1=r1, r2=r2, n1=n1, n2=n2))
    return n2

## Calculate Substance from Moles

In [10]:
MolesStep3 = MolesStep('Step 3: Do something else',
                       'What you want to calculate:')

In [11]:
@MolesStep3.register_conversion_method(
    functionName='Concentration',
    parameters=(('Name',str), ('Moles',float), ('Volume',float))
)
def calculate_concentration(s, n, V):
    '''c=n/V'''
    c = n/V
    print('c({s}) = n/V = {n:.8g} / {V} = {c:.8g}moldm-3'.format(s=s, n=n, V=V, c=c))
    return c

In [12]:
@MolesStep3.register_conversion_method(
    functionName='Solution Volume',
    parameters=(('Name',str), ('Moles',float), ('Concentration',float))
)
def calculate_solution_volume(s, n, c):
    '''V=n/c'''
    V = n/c
    print('V({s}) = n/c = {n:.8g} / {c} = {V:.8g}dm3'.format(s=s, n=n, c=c, V=V))
    return V

In [13]:
@MolesStep3.register_conversion_method(
    functionName='Mass',
    parameters=(('Name',str), ('Moles',float), ('Molecular Mass',float))
)
def calculate_mass(s, n, Mr):
    '''m=n*Mr'''
    m = n*Mr
    print('m({s}) = nxMr = {n:.8g} x {Mr} = {m:.8g}g'.format(s=s, n=n, Mr=Mr, m=m))
    return m

In [14]:
@MolesStep3.register_conversion_method(
    functionName='Gas Volume',
    parameters=(('Name',str), ('Moles',float))
)
def calculate_gas_volume(s, n):
    '''V = 24n at 25°C (298K) and 1atm'''
    V = 24*n
    print('V({s}) = nx24 = {n} x 24 = {V}'.format(s=s, n=n, V=V))
    return V

## User Input

In [15]:
from IPython.display import display, clear_output

In [16]:
display(widgets.HTML(value='<h2>Enter Calculation Data</h2><i>Volume is always in dm3!!!</i>'))
display(MolesPrepStep.make_box())
display(MolesStep1.make_box('Name',))
display(MolesStep2.make_box('Name 1','Name 2','Moles 1'))
display(MolesStep3.make_box('Name','Moles'))

In [19]:
w_button = widgets.Button(description='Run Calculations')
display(w_button)

def handle_run(*args):
    clear_output()
    s1, s2 = MolesPrepStep.run()
    n1 = MolesStep1.run({'Name':s1})
    n2 = MolesStep2.run({'Name 1':s1,'Name 2':s2,'Moles 1':n1})
    final = MolesStep3.run({'Name':s2,'Moles':n2})
w_button.on_click(handle_run)

n(H2SO4) = cV = 0.1 x 0.0215 = 0.00215mol
From eqn 1 mol H2SO4:2 mol NaOH
TF n(NaOH) = 0.00215 x (2/1) = 0.0043mol
c(NaOH) = n/V = 0.0043 / 0.025 = 0.172moldm-3
